In [2]:

#The code is used to fetch a bunch of gene information from Entrez
from Bio import Entrez,SeqIO,SeqFeature
import sys,re,sqlite3
Entrez.email = 'ruanzy@berkeley.edu'
enzyme = ['Hexokinase', 'Glucose-6-phosphate isomerase', 'Triosephosphate isomerase', ''Fructose-bisphosphate aldolase'',\
          'Citrate synthase', 'Aconitase', 'Isocitrate dehydrogenase', 'α-Ketoglutarate dehydrogenase',\
          'glucose 6-phosphate dehydrogenase', '6-phosphogluconolactonase', '6-phosphogluconate dehydrogenase', 'Ribose-5-phosphate isomerase']
species = ['Drosophila melanogaster', 'Escherichia coli', 'Homo sapiens']
with open("gene_info_2.gb.txt", 'w') as local_file: #create a gb file to store the gene info from entrez
    for enz in enzyme:# this loop fetch a handful of gene infomation from the online database
        for spec in species:
            term_con=spec +  '[ORGN] ' + enz
            handle = Entrez.esearch(db='nucleotide',
                                    retmax=1,
                                    term=term_con,
                                    sort='relevance',)
            try:
                id_fetch=Entrez.read(handle)['IdList'][0]
            except IndexError:
                print("No match for query")
            gene_fetch=Entrez.efetch(db="nucleotide", id=id_fetch, rettype="gbwithparts",retmode="text")
            #print(gene_fetch.read())
            #gene_fetch_line=gene_fetch.readlines()
            local_file.write(gene_fetch.read())
gene_fetch.close()   

No match for query
No match for query


previous code will produce a file named **gene_info_2.gb.txt**, which will be parsed by SeqIO later. Advance notice is given here that **my.db** will be created in the following code to store 3 tables

In [9]:
#code is used to create a gene table including id,name,description,organism,chromosome,translated sequence,and nucleotide sequence
import sqlite3
from Bio import Entrez,SeqIO,SeqFeature
#search specific info in gene_info.gb.txt to create a gene table
records = list(SeqIO.parse("gene_info_2.gb.txt", "genbank"))
#set up a new database
i = 0
idofgene=[]
nameofgene=[]
descriptionofgene=[]
organismofgene=[]
while i < len(records):
    idofgene.append(records[i].id)
    nameofgene.append(records[i].name)
    descriptionofgene.append(records[i].description)
    organismofgene.append(records[i].annotations["source"])
    i+=1
chromosome=[]
translated_seq=[]
nucleotide_seq=[]
for record in records:#collect nucleotide sequence 
    nucleotide_seq.append(record.seq)
for record in records:#collect chromosome information
    for (index, feature) in enumerate(record.features) :
        if feature.type=="source" :
            if "chromosome" in feature.qualifiers :
                for value in feature.qualifiers["chromosome"] :
                    chromosome.append(value)
            else:
                chromosome.append("N/A")
for record in records:#collect translation sequence
    for (index, feature) in enumerate(record.features) :
        if feature.type=="CDS" :
            if "translation" in feature.qualifiers :
                for value in feature.qualifiers["translation"] :
                    translated_seq.append(value)
            else:
                translated_seq.append("N/A")
nucleotide_seq_str = ["%s" % member for member in nucleotide_seq]
gene_collection = zip(idofgene,nameofgene,descriptionofgene,organismofgene,chromosome,translated_seq,nucleotide_seq_str)
gene_array=list(gene_collection)
#print(gene_array[1])

# connect database
conn = sqlite3.connect('my.db')
c = conn.cursor()
# create tables: gene, pathway, enzyme
c.execute("""CREATE TABLE genes (id TEXT,
                                name TEXT,
                                description TEXT,
                                organism TEXT,
                                chromosome TEXT,
                                translated_sequence VARCHAR(10000),
                                nucleotide_sequence VARCHAR(10000))""")
c.executemany('INSERT INTO genes VALUES (?,?,?,?,?,?,?)',gene_array)
#c.execute("INSERT INTO genes VALUES (1,'NM_001272455','Drosophila melanogaster hexokinase A, transcript variant C (Hex-A), mRNA','Drosophila melanogaster (fruit fly)')")
conn.commit()
c.execute("SELECT * FROM genes WHERE name = 'GG771780';")
print(c.fetchone()) #this is an example to show how the gene table is like



    
    
    


('GG771780.1', 'GG771780', 'Escherichia coli MS 115-1 genomic scaffold Scfld339, whole genome shotgun sequence', 'Escherichia coli MS 115-1', 'N/A', '...')


to create a pathway table including pathway and description of the pathway 

In [21]:
pathway = ['glycolysis','citric acid cycle','pentose phosphaste']
des_of_path = ['the metabolic pathway that converts glucose into pyruvate, CH3COCOO− and H+',\
               'a series of chemical reactions used by all aerobic organisms to release stored energy through the oxidation of acetyl-CoA derived from carbohydrates, fats, and proteins into adenosine triphosphate (ATP) and carbon dioxide',\
               'a metabolic pathway parallel to glycolysis']
pathway_array=zip(pathway,des_of_path)
pathway_collection=list(pathway_array)
print(pathway_collection)
# connect database
conn = sqlite3.connect('my.db')
c = conn.cursor()

# create tables: gene, pathway, enzyme

c.execute("""CREATE TABLE pathways (name TEXT,
                                description TEXT)""")
c.executemany('INSERT INTO pathways VALUES (?,?)', pathway_collection)
conn.commit()
t = ('glycolysis',)
c.execute('SELECT * FROM pathways WHERE name=?', t)
print(c.fetchone()) #this is an example to show how the gene table is like

[('glycolysis', 'the metabolic pathway that converts glucose into pyruvate, CH3COCOO− and H+'), ('citric acid cycle', 'a series of chemical reactions used by all aerobic organisms to release stored energy through the oxidation of acetyl-CoA derived from carbohydrates, fats, and proteins into adenosine triphosphate (ATP) and carbon dioxide'), ('pentose phosphaste', 'a metabolic pathway parallel to glycolysis')]
('glycolysis', 'the metabolic pathway that converts glucose into pyruvate, CH3COCOO− and H+')


to create a enzyme table including name and function of the enzyme

In [33]:
records = list(SeqIO.parse("gene_info_2.gb.txt", "genbank"))
EC_number=[]
EC_number_true=[]#pick up first EC number at each record
for record in records:#collect translation sequence
    for (index, feature) in enumerate(record.features) :
        if feature.type=="CDS" :
            if "EC_number" in feature.qualifiers :
                for value in feature.qualifiers["EC_number"] :
                    EC_number.append(value)
            else:
                EC_number.append("N/A")
    EC_number_true.append(EC_number[0])#take first EC number for each record as some records may have more than one EC number
    EC_number[:]=[]  
function=['phosphorylates hexoses (six-carbon sugars), forming hexose phosphate',\
         'interconverts glucose-6-phosphate (G6P) and fructose-6-phosphate (F6P)',\
         'catalyzes the reversible interconversion of the triose phosphate isomers dihydroxyacetone phosphate and D-glyceraldehyde 3-phosphate',\
         'catalyzing a reversible reaction that splits the aldol, fructose 1,6-bisphosphate, into the triose phosphates dihydroxyacetone phosphate and glyceraldehyde 3-phosphate',\
         'catalyzes the condensation reaction of the two-carbon acetate residue from acetyl coenzyme A and a molecule of four-carbon oxaloacetate to form the six-carbon citrate',\
         'catalyses the stereo-specific isomerization of citrate to isocitrate via cis-aconitate',\
         'catalyzes the oxidative decarboxylation of isocitrate, producing alpha-ketoglutarate (α-ketoglutarate) and CO2',\
         'α-ketoglutarate + NAD+ + CoA → Succinyl CoA + CO2 + NADH',\
         'D-glucose 6-phosphate + NADP+ ⇌ 6-phospho-D-glucono-1,5-lactone + NADPH + H+',\
         'catalyzes the conversion of 6-phosphogluconolactone to 6-phosphogluconic acid',\
         'catalyses the decarboxylating reduction of 6-phosphogluconate into ribulose 5-phosphate in the presence of NADP',\
         'catalyzes the conversion between ribose-5-phosphate (R5P) and ribulose-5-phosphate (Ru5P)']
enzyme_collection=zip(enzyme,EC_number_true,function)
enzyme_array=list(enzyme_collection)
# connect database
conn = sqlite3.connect('my.db')
c = conn.cursor()

# create tables: gene, pathway, enzyme
'''
c.execute("""CREATE TABLE ENZYME (name TEXT,ec_number TEXT,function TEXT)""")'''
c.executemany('INSERT INTO ENZYME VALUES (?,?,?)', enzyme_array)
conn.commit()
t = ('Hexokinase',)
c.execute('SELECT * FROM ENZYME WHERE name=?', t)
print(c.fetchone()) #this is an example to show how the gene table is like


('Hexokinase', '2.7.1.1', 'phosphorylates hexoses (six-carbon sugars), forming hexose phosphate')
